# Volume 3: MongoDB.
    Darren Lund
    I AM BATMAN
    WHY?

In [7]:
import re
import json
import numpy as np
from pymongo import MongoClient

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [6]:
client = MongoClient()
db = client.db1
col = db.collection1
col.delete_many({})
with open('trump.json','r') as infile :
    for line in infile.readlines() :
        col.insert_one(json.loads(line))
col.count()

95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [8]:
russ = re.compile("russia",re.IGNORECASE)
mention_russia = col.find({'text':{'$regex':russ}})
print(str(mention_russia.count())+" tweets about Russia.")

5841 tweets about Russia.


In [10]:
time_zones = ['Central Time (US & Canada)','Pacific Time (US & Canada)','Eastern Time (US & Canada)','Mountain Time (US & Canada)']
in_us = col.find({'$or':[{'user.time_zone':time_zones[i]} for i in range(len(time_zones))]})
print(str(in_us.count())+" tweets from US and Canadian time zones.")

33939 tweets from US and Canadian time zones.


In [13]:
languages = col.distinct('user.lang')
counts = {}
for language in languages :
    counts[language] = col.find({'user.lang':language}).count()
print(str(len(languages))+" different languages.")
print(counts)

48 different languages.
{'en': 82984, 'en-gb': 756, 'tr': 615, 'fr': 1612, 'es': 3496, 'it': 588, 'pt': 752, 'ru': 779, 'nl': 389, 'hr': 24, 'da': 49, 'pl': 123, 'ja': 91, 'id': 95, 'de': 1024, 'el': 33, 'th': 15, 'uk': 4, 'en-GB': 93, 'sv': 121, 'ar': 95, 'ca': 47, 'cs': 13, 'no': 19, 'pt-PT': 1, 'fi': 36, 'en-AU': 3, 'ko': 15, 'ro': 4, 'he': 11, 'fa': 8, 'eu': 1, 'lv': 1, 'vi': 6, 'xx-lc': 5, 'hi': 1, 'zh-CN': 9, 'zh-cn': 5, 'es-MX': 2, 'zh-TW': 2, 'hu': 6, 'sr': 6, 'zh-Hans': 1, 'sk': 1, 'gl': 2, 'zh-tw': 2, 'af': 1, 'en-US': 1}


## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [23]:
populars = col.find({'user.followers_count':{'$gt':1000000}}).sort('user.followers_count',-1)
count = 0
unique_ids = []
print("Most followed Tweeters :")
for person in populars :
    if count < 5 and person['user']['id'] not in unique_ids :
        unique_ids.append(person['user']['id'])
        print(person['user']['screen_name'] + '\twith ' + str(person['user']['followers_count']) + ' followers.')
        count += 1
    elif count >= 5 :
        break

Most followed Tweeters :
CNN	with 35481010 followers.
Reuters	with 17821353 followers.
FoxNews	with 14663497 followers.
WSJ	with 14023091 followers.
TIME	with 14017299 followers.


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [26]:
col.update_many({},{'$unset':{'retweeted_status':""}})
col.update_many({'user.followers_count':{'$gte':1000}},{'$set':{'popular':True}})
print(str(col.find({'popular':True}).count())+" popular tweets.")

32489 popular tweets.
